In [1]:
# Imports 
import os
import time
import datetime
import scipy.misc
import shutil
import numpy as np
import posixpath
import sys
sys.path.insert( 0,os.path.abspath(os.path.join(os.getcwd(), '..')))

from breadboard import BreadboardClient
bc = BreadboardClient(config_path='API_CONFIG.json', lab_name='fermi1')


import warnings
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")
warnings.filterwarnings("ignore", "Could not find appropriate MS Visual C Runtime")


# Helper Functions

In [2]:
def select_today(main_folder="\\\\bec1server\\Raw Data\\Fermi3\\Images"):
    '''
    Get today's folder
    ===============================
    '''
    start_time = datetime.datetime.now() 
    start_year = start_time.strftime('%Y')
    start_month = start_time.strftime('%Y-%m')
    start_date = start_time.strftime('%Y-%m-%d')
    output_folder = os.path.join(main_folder, start_year, start_month, start_date)
    if not (os.path.exists(output_folder)): os.makedirs(output_folder)
    return output_folder


def getFileList(folder = 'Not Provided'):
    '''
    Get a list of files in a folder
    ===============================
    '''
    # Confirm that given folder path is correct
    if not os.path.exists(folder): raise ValueError("Folder '{}' doesn't exist".format(folder))
    # Folder contents
    filenames = [filename for filename in os.listdir(folder)]
    # Output
    paths = [os.path.join(folder,f) for f in filenames]
    return (filenames, paths)

# Main Code

In [3]:
# Global settings 
refresh_time = 0.5 # seconds
main_folder = "/Users/biswaroopmukherjee/Documents/Physics/Research/Zwierlein/testfolder"


In [29]:
watchfolder  = select_today(main_folder)
names_old, paths_old = getFileList(watchfolder)

# Main Loop 
while True: 
    # set watchfolder: in this case, it's for example "main_folder/2020/2020-02/2020-02-21" if today was 2020-02-21
    watchfolder = select_today(main_folder)
    # Get a list of all the images in the folder 
    names, paths = getFileList(today_folder)
    names = sorted(names)
    paths = sorted(paths)
    
    # check if a new image has come in
    if len(names)>len(names_old):
        output_filename = names[-1]
        output_filepath = paths[-1]
        dt = datetime.datetime.fromtimestamp(os.path.getctime(output_filepath))
        
        # Write to Breadboard
        try:
            resp = bc.post_images(
                        image_names = os.path.splitext(output_filename)[0],
                        filepath = output_filepath,
                        image_times = [dt], 
                        auto_time = False
                        )
            if resp.status_code!=200:
                print(resp.text)
        except: pass
        names_old = names

    # Wait before checking again 
    time.sleep(refresh_time)

02-10-2020_13_26_07.fits
2020-02-21 14:37:50.934848


KeyboardInterrupt: 

# Tests